# Quantum Algorithm for a Controlled Interferometer
 In this notebook, the [QISKit](https://github.com/QISKit) is used to simulate a Mach-Zehnder Interferometer, with the   results being plotted using Matplotlib.
 Notice that the variables names and comments are in english, but the graphs label are in portuguese.

In [ ]:
###Imports
#QISKit for the simulation
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, register
#Numpy for Linear Algebra
import numpy as np
#Pandas for dataframes
import pandas as pd
#Matplotlib for plotting and graphs
import matplotlib.pyplot as plt
#Basic Units for enabling the use of radians in the plot axis
import basic_units as bau
#Enabling the plots appearing inside the notebook
%matplotlib inline

In [ ]:
#Defining a function for running the experiments, given a quantum circuit object,
#a float for alpha, an array with phases, and the number of tests.
#This function returns a dataframe with the results of the tests.
def run_experiments(qc, alpha = 0, phases = [0, np.pi, 2*np.pi], number_of_tests = 1000):
    _df = pd.DataFrame(0, index = phases, columns=['00', '01', '10', '11'])
    
    for phase in phases:
        qc.reset(q)
        qc.h(q[0]); qc.h(q[1])

        qc.u1(alpha, q[0]); qc.u1(phase, q[1])
        
        qc.h(q[0]); qc.ch(q[0],q[1])

        qc.measure(q, c)
        job_sim = execute(qc, "local_qasm_simulator", shots = number_of_tests)
        sim_result = job_sim.result()

        if '00' in sim_result.get_counts(qc): _df.loc[phase, '00'] = sim_result.get_counts(qc)['00']
        if '01' in sim_result.get_counts(qc): _df.loc[phase, '01'] = sim_result.get_counts(qc)['01'] 
        if '10' in sim_result.get_counts(qc): _df.loc[phase, '10'] = sim_result.get_counts(qc)['10']         
        if '11' in sim_result.get_counts(qc): _df.loc[phase, '11'] = sim_result.get_counts(qc)['11']
    
    
    return _df/number_of_tests

#Defining a routine that runs the experiments (using the run_experiments function) and then makes a plot.
def run_experiments_and_plot(qc, alpha = 0, phases = [0, np.pi, 2*np.pi], number_of_tests = 1000):
    _df = run_experiments(qc, alpha, phases, number_of_tests)
    
    plt.figure()
    point_size = 10
    ax = _df['00'].plot(style = 'o', legend=True, figsize = (10,5), ms = point_size, label = r'$|00 \rangle$')
    _df['01'].plot(style = 'x', legend=True, ms = point_size, label = r'$|01 \rangle$')
    _df['10'].plot(style = '+', legend=True, ms = point_size, label = r'$|10 \rangle$')
    _df['11'].plot(style = '*', legend=True, ms = point_size, label = r'$|11 \rangle$')

    x = [1*bau.radians]
    ax.plot(x, 0, xunits=bau.radians)

    plt.title('Resultados para o interferômetro controlado (' + str(number_of_tests) + ' experimentos por fase, ' + r'$\alpha = %.2f \pi)$' %(alpha/np.pi))
    plt.xlabel('Fase aplicada')
    plt.ylabel('Frequência do resultado')
    plt.grid()

    plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

In [ ]:
#Creating a quantum circuit with two qubits
q = QuantumRegister(2)
c = ClassicalRegister(2)
qc = QuantumCircuit(q, c)

#Creating a array of values for the phase shift of the qubit
phases = list(np.linspace(0, 1, 40) * 4*np.pi)
#Creating a array for storing the values for alpha
values_for_alpha = list(np.linspace(0, 1, 5) * 2*np.pi)
#Creating a variable for holding the number of simulations to be made
number_of_tests = 500

#Notice that the bigger the alpha array, phases array, and the number of tests are,
#the longer it will take for the code to finish.

In [ ]:
#Iterate over each alpha value, making experiments and plotting the results
for a in values_for_alpha: run_experiments_and_plot(qc, a, phases, number_of_tests)